# Traveling Salesman Problem
<img src="TSP.png">
Soltion to 48 States TSP. 

**Given a list of cities n and the distances between each pair of cities, what is the shortest possible route that visits each city and returns to the origin city?**

ref: http://www.opl.ufc.br/post/tsp/

## Input
1. The (x,y) coordinates of each city.
2. $Cost_{ij}$: the distance between city i to city j.


## What's output?
A sequnce of city to visit, like 1->10->3->28->5....->1

## Variables
$$
x_{ij} = \begin{cases}
&1 \quad \text{ if the route includes a direct link between cities } i \text{ and } j,  \\
&0 \quad \text{otherwise}.
\end{cases}
$$

## Objective
minimize the total travel distance: 
$$\text{min} \sum_i \sum_j c_{ij}x_{ij}$$

## Constraint1
each city is arrived at from exactly one other city ---- **One In**

$$
\sum_{i=1, i \neq j}^{n} x_{ij} = 1, \quad j=1,2,…,n,\\
$$

## Constraint2
from each city there is a departure to exactly one next city. ----**One Out**
$$
 \sum_{j=1, j \neq i}^{n} x_{ij} = 1, \quad i=1,2,…,n,\\
$$

Subtour

<img src="Subtour1.png">

The first subtour goes through nodes Toledo, Cincy, and Dayton, while the second subtour goes through nodes Clevland and Akron.

Note that in this solution each node has exactly two edges in the tour incident to it. But there is no path for the salesman to travel between subtour. 

So we must add extra constraints to our model to eliminate these solutions.




## Subtour constraint: Miller-Tucker-Zemlin formulation
Let's define a variable $\mu_i = t$ to represent city i is visited in step t(t=1,2,...,n) of the tour.  $\mu_{Toledo} =1$, $\mu_{Cincy} =2$, $\mu_{Dayton} =3$, $\mu_{Cleveland} =4$, $\mu_{Akron} = 5.$


## $\mu_i - \mu_j + nx_{ij}\le n-1,  2 \leq i \neq j \leq n$


### Subtour:

1. Toledo -> Cincy -> Dayton -> Toledo

2. Cleveland -> Arkon -> Cleveland

#### subtour1:
$$
\begin{align}
& A: \mu_{Toledo} - \mu_{Cincy} + 5 * x_{Todeldo, Cincy} \le 4 \\
& B: \mu_{Cincy} - \mu_{Dayton} + 5 * x_{Cincy, Dayton} \le 4 \\
& A+B:  \mu_{Toledo} - \mu_{Dayton} + 10 \le 8 \\
\end{align}
$$
It's always possible to define $ \mu_{Toledo}$, $ \mu_{Dayton}$ to satisfy this. 

#### subtour2:
$$
\begin{align}
& A: \mu_{Akron}- \mu_{Cleveland} + 5 * x_{Clevend,Akron} \le 4 \\
& B: \mu_{Cleveland}- \mu_{Akron} + 5 * x_{Akron, Clevend} \le 4 \\
& A+B: \quad 10 \le 8
\end{align}
$$


## tour 
<img src="tour.png">
a feasible tour is  Toledo->Cincy->Dayton->Arkon->Cleveland->Toledo


$$
\begin{align}
& A: \mu_{Toledo}- \mu_{Cincy} + 5 *  x_{Todeldo, Cincy} \le 4 \\
& B: \mu_{Cincy} - \mu_{Dayton} + 5 * x_{Cincy, Dayton} \le 4\\
& C: \mu_{Dayton}- \mu_{Arkon} + 5 * x_{Dayton,Arkon} \le 4  \\
& D: \mu_{Akron}- \mu_{Cleveland} + 5 * x_{Akron,Clevend} \le 4 \\ 
& A+B+C+D: \mu_{Toledo} - \mu_{Clevend} +20 \le 16
\end{align}
$$

It's always possible to find $\mu_{Toledo}$,  $\mu_{Clevend}$ to satisfiy the whole tour. 



### The TSP in Integer linear programming formulations
$$
\begin{align}
\text{min} \quad & \sum_{i = 1}^{n} \sum_{j = 1, j \neq i}^{n} c_{ij} x_{ij}, \\
\text{subject to} \\
& \sum_{i=1, i \neq j}^{n} x_{ij} = 1, \quad j=1,2,...,n,\\
& \sum_{j=1, j \neq i}^{n} x_{ij} = 1, \quad i=1,2,...,n,\\
& u_i - u_j + n x_{ij} \leq n-1, \quad 2 \leq i \neq j \leq n, \\
& x_{ij} \in \{0,1\} \quad i, j =1,2,...,n, \quad i \neq j, \\
& u_i \in \mathbb{R}^+ \quad i=1,2,...,n \\
\end{align}
$$


### Code

In [1]:
cost_matrix = []
file = open('17.txt')
lines = file.readlines()
file.close()

for i in range(len(lines)):
    aux = lines[i][:-1].split('\t')
    aux = [int(i) for i in aux if i!= '']
    cost_matrix.append(aux)

n = len(cost_matrix)

In [2]:
cost_matrix


[[9999, 3, 5, 48, 48, 8, 8, 5, 5, 3, 3, 0, 3, 5, 8, 8, 5],
 [3, 9999, 3, 48, 48, 8, 8, 5, 5, 0, 0, 3, 0, 3, 8, 8, 5],
 [5, 3, 9999, 72, 72, 48, 48, 24, 24, 3, 3, 5, 3, 0, 48, 48, 24],
 [48, 48, 74, 9999, 0, 6, 6, 12, 12, 48, 48, 48, 48, 74, 6, 6, 12],
 [48, 48, 74, 0, 9999, 6, 6, 12, 12, 48, 48, 48, 48, 74, 6, 6, 12],
 [8, 8, 50, 6, 6, 9999, 0, 8, 8, 8, 8, 8, 8, 50, 0, 0, 8],
 [8, 8, 50, 6, 6, 0, 9999, 8, 8, 8, 8, 8, 8, 50, 0, 0, 8],
 [5, 5, 26, 12, 12, 8, 8, 9999, 0, 5, 5, 5, 5, 26, 8, 8, 0],
 [5, 5, 26, 12, 12, 8, 8, 0, 9999, 5, 5, 5, 5, 26, 8, 8, 0],
 [3, 0, 3, 48, 48, 8, 8, 5, 5, 9999, 0, 3, 0, 3, 8, 8, 5],
 [3, 0, 3, 48, 48, 8, 8, 5, 5, 0, 9999, 3, 0, 3, 8, 8, 5],
 [0, 3, 5, 48, 48, 8, 8, 5, 5, 3, 3, 9999, 3, 5, 8, 8, 5],
 [3, 0, 3, 48, 48, 8, 8, 5, 5, 0, 0, 3, 9999, 3, 8, 8, 5],
 [5, 3, 0, 72, 72, 48, 48, 24, 24, 3, 3, 5, 3, 9999, 48, 48, 24],
 [8, 8, 50, 6, 6, 0, 0, 8, 8, 8, 8, 8, 8, 50, 9999, 0, 8],
 [8, 8, 50, 6, 6, 0, 0, 8, 8, 8, 8, 8, 8, 50, 0, 9999, 8],
 [5, 5, 26, 12, 12, 

In [3]:
import pandas as pd
cost_matrix = pd.read_csv("17.txt", sep="\t",   header=None)
cost_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,9999,3,5,48,48,8,8,5,5,3,3,0,3,5,8,8,5
1,3,9999,3,48,48,8,8,5,5,0,0,3,0,3,8,8,5
2,5,3,9999,72,72,48,48,24,24,3,3,5,3,0,48,48,24
3,48,48,74,9999,0,6,6,12,12,48,48,48,48,74,6,6,12
4,48,48,74,0,9999,6,6,12,12,48,48,48,48,74,6,6,12
5,8,8,50,6,6,9999,0,8,8,8,8,8,8,50,0,0,8
6,8,8,50,6,6,0,9999,8,8,8,8,8,8,50,0,0,8
7,5,5,26,12,12,8,8,9999,0,5,5,5,5,26,8,8,0
8,5,5,26,12,12,8,8,0,9999,5,5,5,5,26,8,8,0
9,3,0,3,48,48,8,8,5,5,9999,0,3,0,3,8,8,5


In [4]:
from pyomo.environ import *
n = len(cost_matrix) # total number of city 

$$
\begin{align}
\text{min} \quad & \sum_{i = 1}^{n} \sum_{j = 1, j \neq i}^{n} c_{ij} x_{ij}, \\
\text{subject to} \\
& \sum_{i=1, i \neq j}^{n} x_{ij} = 1, \quad j=1,2,...,n,\\
& \sum_{j=1, j \neq i}^{n} x_{ij} = 1, \quad i=1,2,...,n,\\
& u_i - u_j + n x_{ij} \leq n-1, \quad 2 \leq i \neq j \leq n, \\
& x_{ij} \in \{0,1\} \quad i, j =1,2,...,n, \quad i \neq j, \\
& u_i \in \mathbb{R}^+ \quad i=1,2,...,n \\
\end{align}
$$


In [5]:
columns = cost_matrix.columns.astype(str)
cost_matrix.columns = cost_matrix.columns.map(str)
cost_matrix.index   = cost_matrix.index.map(str)

In [6]:
model = ConcreteModel()

In [7]:
model.city = Set(initialize = cost_matrix.columns)


In [23]:
cost_matrix1 = cost_matrix.stack().to_dict()
cost_matrix1

{('0', '0'): 9999,
 ('0', '1'): 3,
 ('0', '2'): 5,
 ('0', '3'): 48,
 ('0', '4'): 48,
 ('0', '5'): 8,
 ('0', '6'): 8,
 ('0', '7'): 5,
 ('0', '8'): 5,
 ('0', '9'): 3,
 ('0', '10'): 3,
 ('0', '11'): 0,
 ('0', '12'): 3,
 ('0', '13'): 5,
 ('0', '14'): 8,
 ('0', '15'): 8,
 ('0', '16'): 5,
 ('1', '0'): 3,
 ('1', '1'): 9999,
 ('1', '2'): 3,
 ('1', '3'): 48,
 ('1', '4'): 48,
 ('1', '5'): 8,
 ('1', '6'): 8,
 ('1', '7'): 5,
 ('1', '8'): 5,
 ('1', '9'): 0,
 ('1', '10'): 0,
 ('1', '11'): 3,
 ('1', '12'): 0,
 ('1', '13'): 3,
 ('1', '14'): 8,
 ('1', '15'): 8,
 ('1', '16'): 5,
 ('2', '0'): 5,
 ('2', '1'): 3,
 ('2', '2'): 9999,
 ('2', '3'): 72,
 ('2', '4'): 72,
 ('2', '5'): 48,
 ('2', '6'): 48,
 ('2', '7'): 24,
 ('2', '8'): 24,
 ('2', '9'): 3,
 ('2', '10'): 3,
 ('2', '11'): 5,
 ('2', '12'): 3,
 ('2', '13'): 0,
 ('2', '14'): 48,
 ('2', '15'): 48,
 ('2', '16'): 24,
 ('3', '0'): 48,
 ('3', '1'): 48,
 ('3', '2'): 74,
 ('3', '3'): 9999,
 ('3', '4'): 0,
 ('3', '5'): 6,
 ('3', '6'): 6,
 ('3', '7'): 12,
 ('3',

In [9]:
model.cost = Param(model.city, model.city, initialize = cost_matrix1)

In [10]:
### Varibles
model.x = Var(model.city, model.city, domain = Binary)

In [11]:
### objective
def obj_func(model):
    return summation(model.cost, model.x)

model.obj = Objective(rule = obj_func, sense=minimize)

In [12]:
### Constraints
#1. one city in
def _one_in(model, in_city):
    return sum(model.x[i, in_city] for i in model.city if i != in_city)  == 1

model.cons_one_in = Constraint(model.city, rule = _one_in)

In [13]:
### Constraints
#1. one city out
def _one_out(model, out_city):
    return sum(model.x[out_city, j ] for j in model.city if j != out_city)  == 1

model.cons_one_out = Constraint(model.city, rule = _one_out)

In [14]:
# auxilary variables
model.U = Set(initialize = cost_matrix.columns.drop('0'))
model.u = Var(model.city, 
              domain = NonNegativeIntegers, 
              bounds = (0, n-1) )

In [15]:
def _subtour(model,i,j):
    if i!=j: 
        return model.u[i] - model.u[j] + model.x[i,j] * n <= n-1
    else:
        #Yeah, this else doesn't say anything
        return model.u[i] - model.u[i] == 0 
    
model.subtour = Constraint(model.U,model.city,rule=_subtour)

In [16]:
solver = SolverFactory('glpk')
result = solver.solve(model, tee= True, timelimit = 300)
# solver = SolverFactory('gurobi', solver_io="python")
# result = solver.solve(model, tee= True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --tmlim 300 --write C:\Users\kon6750\AppData\Local\Temp\tmpxc2t_wrf.glpk.raw
 --wglp C:\Users\kon6750\AppData\Local\Temp\tmpmp9gs5w6.glpk.glp --cpxlp C:\Users\kon6750\AppData\Local\Temp\tmp_6wp2in3.pyomo.lp
Reading problem data from 'C:\Users\kon6750\AppData\Local\Temp\tmp_6wp2in3.pyomo.lp'...
C:\Users\kon6750\AppData\Local\Temp\tmp_6wp2in3.pyomo.lp:2836: warning: lower bound of variable 'x1' redefined
C:\Users\kon6750\AppData\Local\Temp\tmp_6wp2in3.pyomo.lp:2836: warning: upper bound of variable 'x1' redefined
307 rows, 307 columns, 1313 non-zeros
306 integer variables, 289 of which are binary
3125 lines were read
Writing problem data to 'C:\Users\kon6750\AppData\Local\Temp\tmpmp9gs5w6.glpk.glp'...
2492 lines were written
GLPK Integer Optimizer, v4.65
307 rows, 307 columns, 1313 non-zeros
306 integer variables, 289 of which are binary
Preprocessing...
290 rows, 289 columns, 1312 non-zeros
289 integer variab

In [17]:
print(result)


Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 307
  Number of variables: 307
  Number of nonzeros: 1313
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: feasible
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 300.24454283714294
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [18]:
l = list(model.x.keys())
for i in l:
    if model.x[i]() != 0:
        print(i,'--', model.x[i]())
print(model.obj())

('0', '2') -- 1.0
('1', '10') -- 1.0
('2', '13') -- 1.0
('3', '4') -- 1.0
('4', '6') -- 1.0
('5', '14') -- 1.0
('6', '7') -- 1.0
('7', '16') -- 1.0
('8', '11') -- 1.0
('9', '12') -- 1.0
('10', '15') -- 1.0
('11', '0') -- 1.0
('12', '1') -- 1.0
('13', '9') -- 1.0
('14', '3') -- 1.0
('15', '5') -- 1.0
('16', '8') -- 1.0
41.0


The 